https://github.com/apache/spark/tree/master/examples/src/main/python

https://datascience-enthusiast.com/Python/pythonindex.html

In [5]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f



In [6]:
from pyspark.sql import Row
import datetime
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

1 - joindre les dataframe sur la clé ID_CLIENT en concervant tous les clients du périmètre.
2 - compter le nombre de ID_CLIENT et vérifier qu'il correspond aux nombre d'ID_CLIENT dans la variable perimètre.
3 - Caster les variables continues en double et sauvergarder alors le df obtenu dans le repertoire data sur le cluster.
4 - Pour les variables catégorielles, créer une nouvelle variable qui prend la modalité de la variable courante si elle existe et "NA" sinon.
5- Verifier la cohérence des variables continue. Par exemple pour une variable comme age mettre à -1 tous les ages <0 ou>120ans

In [7]:
perimetre = spark_session.read.csv("data/sample_perimetre.csv", header=True)
histo_client_raw = spark_session.read.csv("data/sample_histo_client.csv", header=True)
histo_train_raw = spark_session.read.csv("data/sample_histo_train.csv", header=True)
histo_lowcost_raw = spark_session.read.csv("data/sample_histo_lowcost.csv", header=True)
visites_raw = spark_session.read.csv("data/sample_visites.csv", header=True)

In [8]:
#histo_client_raw.select("LBL_SEG_COMPORTEMENTAL").distinct().collect()

## Fonction pour recuperer la dimension et les variables d'un dataframe

*Fonction qui permet d'avoir la taille d'un dataframe ainsi que les différentes variables en son sein*

In [9]:
def dtail_data(data):
    print ("Taille du DataFrame : {}".format((data.count(),len(data.columns))))
    return data #affiche les colonnes du dataframe

dtail_data(histo_client_raw)

Taille du DataFrame : (1084162, 11)


DataFrame[ID_CLIENT: string, anciennete: string, recence_cmd: string, AGE: string, LBL_STATUT_CLT: string, LBL_GEO_AIR: string, LBL_GRP_SEGMENT_NL: string, LBL_SEG_COMPORTEMENTAL: string, LBL_GEO_TRAIN: string, LBL_SEGMENT_ANTICIPATION: string, FLG_CMD_CARTE_1225: string]

  ## Transformations sur les dataframes,conversions des variables

*Ecrire une fonction pour transformer les features quantitatives ("anciennete", "recence_cmd", "AGE", etc..) en float*

In [10]:
client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225']

client_col_to_cast=["anciennete","recence_cmd","AGE"]

def cast_columns_of_df(df, cols_to_cast, col_to_keep, cast_type='double'):
    """cast continuous columns into double since all columns are """
    return df.select(col_to_keep + [(df[feature].cast('double'))
                    for feature in cols_to_cast if 'ID_CLIENT' not in feature])


In [11]:
cast_columns_of_df(histo_client_raw,client_col_to_cast,client_cols_to_keep).head(1)

[Row(ID_CLIENT='000843db32fbaecfbb047ca0bb04b1f9f4d9425a', LBL_STATUT_CLT='Grand', LBL_GEO_AIR='Aéroports de Paris Orly', LBL_SEG_COMPORTEMENTAL='Chasseurs Bons Plans', LBL_GEO_TRAIN='Paris', LBL_GRP_SEGMENT_NL='Spectateur', LBL_SEGMENT_ANTICIPATION='Mixte', FLG_CMD_CARTE_1225='0', anciennete=1550.0, recence_cmd=36.0, AGE=None)]

In [12]:
client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225']

def cast_columns_of_df(df, cols_to_cast, col_to_keep, cast_type='double'):
    """cast continuous columns into double since all columns are """
    return df.select(col_to_keep + [(df[feature].cast('double'))
                    for feature in cols_to_cast if 'ID_CLIENT' not in feature])

histo_train = cast_columns_of_df(histo_train_raw, histo_train_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')
histo_lowcost = cast_columns_of_df(histo_lowcost_raw, histo_lowcost_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')

visites = cast_columns_of_df(visites_raw, visites_raw.columns,
                             ["ID_CLIENT"], cast_type='double')

histo_client = cast_columns_of_df(histo_client_raw,
                                  ["anciennete", "recence_cmd", "AGE"],
                                  client_cols_to_keep,
                                 cast_type='double')


 ### Faire une jointure entre les informations des différentes tables. NB: on conservera tous les clients de la table perimetre. En effet, ce sont les cleints qu'on souhaite scorer

In [13]:
def join_by_dataframe (args ,attribut_de_jointure , type_de_jointure):
    data = args[0]
    for i in list(range(1,len(args))):
        data = data.join(args[i] ,on  = attribut_de_jointure , how  = type_de_jointure )
    return data

In [14]:
data3 = join_by_dataframe([perimetre,histo_train,histo_lowcost , visites, histo_client]
                          , attribut_de_jointure = 'ID_CLIENT' ,
                          type_de_jointure = 'left_outer')
datafin = data3.toPandas()
datafin.head(3)

,ID_CLIENT,nb_od,mean_nb_passagers,mean_duree_voyage,mean_mt_voyage,mean_tarif_loisir,mean_classe_1,mean_pointe,mean_depart_we,flg_cmd_lowcost,...,LBL_STATUT_CLT,LBL_GEO_AIR,LBL_SEG_COMPORTEMENTAL,LBL_GEO_TRAIN,LBL_GRP_SEGMENT_NL,LBL_SEGMENT_ANTICIPATION,FLG_CMD_CARTE_1225,anciennete,recence_cmd,AGE
0,000843db32fbaecfbb047ca0bb04b1f9f4d9425a,1.0,1.0,274.666667,58.666667,0.0,0.0,0.00,0.00,NaN,...,Grand,Aéroports de Paris Orly,Chasseurs Bons Plans,Paris,Spectateur,Mixte,0,1550.0,36.0,NaN
1,001338752ea32d9de129c8f8bdf3e2224cf0bd71,1.0,1.0,232.000000,128.200000,1.0,1.0,0.00,0.00,NaN,...,Grand,Aéroport de Marseille Provence (MRS),Comportement Pro,Marseille,Spectateur,Anticipateur,0,1667.0,25.0,35.0
2,003fb9dca8de374386d0fa97b570950583111931,3.0,1.5,248.250000,46.475000,0.5,0.0,0.25,0.25,1.0,...,Moyen moins,Aéroport de Lyon - Saint Exupéry,Rythmes scolaires,Lyon,Spectateur,Peu Anticipateur,1,395.0,15.0,25.0


* Verification de la taille de **perimetre** et le **New dataframe** apres jointure

In [15]:
perimetre.count() == data3.count()

True

*Le jeu de données data3 contient 24 variables  et 1084217 observation*

In [18]:
dtail_data(data3)

Taille du DataFrame : (1084217, 24)


DataFrame[ID_CLIENT: string, nb_od: double, mean_nb_passagers: double, mean_duree_voyage: double, mean_mt_voyage: double, mean_tarif_loisir: double, mean_classe_1: double, mean_pointe: double, mean_depart_we: double, flg_cmd_lowcost: double, flg_track_nl_lowcost: double, flg_track_nl: double, days_since_last_visit: double, tx_conversion: double, LBL_STATUT_CLT: string, LBL_GEO_AIR: string, LBL_SEG_COMPORTEMENTAL: string, LBL_GEO_TRAIN: string, LBL_GRP_SEGMENT_NL: string, LBL_SEGMENT_ANTICIPATION: string, FLG_CMD_CARTE_1225: string, anciennete: double, recence_cmd: double, AGE: double]

In [19]:
data3.columns

['ID_CLIENT',
 'nb_od',
 'mean_nb_passagers',
 'mean_duree_voyage',
 'mean_mt_voyage',
 'mean_tarif_loisir',
 'mean_classe_1',
 'mean_pointe',
 'mean_depart_we',
 'flg_cmd_lowcost',
 'flg_track_nl_lowcost',
 'flg_track_nl',
 'days_since_last_visit',
 'tx_conversion',
 'LBL_STATUT_CLT',
 'LBL_GEO_AIR',
 'LBL_SEG_COMPORTEMENTAL',
 'LBL_GEO_TRAIN',
 'LBL_GRP_SEGMENT_NL',
 'LBL_SEGMENT_ANTICIPATION',
 'FLG_CMD_CARTE_1225',
 'anciennete',
 'recence_cmd',
 'AGE']

In [ ]:
#data1 = perimetre.join(histo_train,on = "ID_CLIENT",how = "left_outer")
#data2 = data1.join(histo_lowcost,on = "ID_CLIENT",how = "left_outer")
#data3 = data2.join(visites,on = "ID_CLIENT",how = "left_outer").join(histo_client,on = "ID_CLIENT",how = "left_outer")
#datafin = data3.toPandas()

### Combien a t'on de features quatitatives, qualitatives ?

*Fonction qui permet d'obtenir le nombre et les différentes variable et  qualitative,quantitative*

In [20]:
data3.dtypes

[('ID_CLIENT', 'string'),
 ('nb_od', 'double'),
 ('mean_nb_passagers', 'double'),
 ('mean_duree_voyage', 'double'),
 ('mean_mt_voyage', 'double'),
 ('mean_tarif_loisir', 'double'),
 ('mean_classe_1', 'double'),
 ('mean_pointe', 'double'),
 ('mean_depart_we', 'double'),
 ('flg_cmd_lowcost', 'double'),
 ('flg_track_nl_lowcost', 'double'),
 ('flg_track_nl', 'double'),
 ('days_since_last_visit', 'double'),
 ('tx_conversion', 'double'),
 ('LBL_STATUT_CLT', 'string'),
 ('LBL_GEO_AIR', 'string'),
 ('LBL_SEG_COMPORTEMENTAL', 'string'),
 ('LBL_GEO_TRAIN', 'string'),
 ('LBL_GRP_SEGMENT_NL', 'string'),
 ('LBL_SEGMENT_ANTICIPATION', 'string'),
 ('FLG_CMD_CARTE_1225', 'string'),
 ('anciennete', 'double'),
 ('recence_cmd', 'double'),
 ('AGE', 'double')]

In [21]:
continuous = []
quali = ["flg_cmd_lowcost"]
def get_quali_quant(data):
    quantitative = 0
    qualitative = 0
    data = data.dtypes
    for i in data:
        if i[1] ==  "double":
            continuous.append(i[0])
            quantitative += 1
        else :
            quali.append(i[0])
            qualitative += 1
    print ("Nbr de variables qualitatives: {} \n Variables qualitatives :{} \n \n Nbr de variables quatitatives: {} \n Variables quantitatives: {}".format(qualitative,quali,quantitative,continuous))
    
get_quali_quant(data3)

Nbr de variables qualitatives: 8 
 Variables qualitatives :['flg_cmd_lowcost', 'ID_CLIENT', 'LBL_STATUT_CLT', 'LBL_GEO_AIR', 'LBL_SEG_COMPORTEMENTAL', 'LBL_GEO_TRAIN', 'LBL_GRP_SEGMENT_NL', 'LBL_SEGMENT_ANTICIPATION', 'FLG_CMD_CARTE_1225'] 
 
 Nbr de variables quatitatives: 16 
 Variables quantitatives: ['nb_od', 'mean_nb_passagers', 'mean_duree_voyage', 'mean_mt_voyage', 'mean_tarif_loisir', 'mean_classe_1', 'mean_pointe', 'mean_depart_we', 'flg_cmd_lowcost', 'flg_track_nl_lowcost', 'flg_track_nl', 'days_since_last_visit', 'tx_conversion', 'anciennete', 'recence_cmd', 'AGE']


### Quelles sont les differentes modalites de la feature LBL_STATUT_CLT ?

*Fonction qui permet d'afficher les différentes modalités d'un dataframa*

In [22]:
def modalite(data,var):
    return data.select(var).distinct().collect()

modalite(data3,"LBL_STATUT_CLT")

[Row(LBL_STATUT_CLT='Moyen moins'),
 Row(LBL_STATUT_CLT='Non present dans la base a cette date'),
 Row(LBL_STATUT_CLT='Nouveau prospect'),
 Row(LBL_STATUT_CLT='Prospect'),
 Row(LBL_STATUT_CLT='Tres petit'),
 Row(LBL_STATUT_CLT=None),
 Row(LBL_STATUT_CLT='Petit'),
 Row(LBL_STATUT_CLT='Inactif'),
 Row(LBL_STATUT_CLT='Nouveau actif'),
 Row(LBL_STATUT_CLT='Grand'),
 Row(LBL_STATUT_CLT='Tres grand'),
 Row(LBL_STATUT_CLT='Moyen plus')]

 * Sauvegarde du dataframe

In [23]:
#data3.write.save("data/data3.csv",format="csv")

In [18]:
continuous

['nb_od',
 'mean_nb_passagers',
 'mean_duree_voyage',
 'mean_mt_voyage',
 'mean_tarif_loisir',
 'mean_classe_1',
 'mean_pointe',
 'mean_depart_we',
 'flg_cmd_lowcost',
 'flg_track_nl_lowcost',
 'flg_track_nl',
 'days_since_last_visit',
 'tx_conversion',
 'anciennete',
 'recence_cmd',
 'AGE']

### Quelles sont les features avec valeurs manquantes remplacer les valeurs manquantes par -1 pour toutes les features ?

In [24]:
# Number of columns
list(enumerate(data3.columns))

[(0, 'ID_CLIENT'),
 (1, 'nb_od'),
 (2, 'mean_nb_passagers'),
 (3, 'mean_duree_voyage'),
 (4, 'mean_mt_voyage'),
 (5, 'mean_tarif_loisir'),
 (6, 'mean_classe_1'),
 (7, 'mean_pointe'),
 (8, 'mean_depart_we'),
 (9, 'flg_cmd_lowcost'),
 (10, 'flg_track_nl_lowcost'),
 (11, 'flg_track_nl'),
 (12, 'days_since_last_visit'),
 (13, 'tx_conversion'),
 (14, 'LBL_STATUT_CLT'),
 (15, 'LBL_GEO_AIR'),
 (16, 'LBL_SEG_COMPORTEMENTAL'),
 (17, 'LBL_GEO_TRAIN'),
 (18, 'LBL_GRP_SEGMENT_NL'),
 (19, 'LBL_SEGMENT_ANTICIPATION'),
 (20, 'FLG_CMD_CARTE_1225'),
 (21, 'anciennete'),
 (22, 'recence_cmd'),
 (23, 'AGE')]

In [25]:
continuous.remove("flg_cmd_lowcost")

In [21]:
continuous

['nb_od',
 'mean_nb_passagers',
 'mean_duree_voyage',
 'mean_mt_voyage',
 'mean_tarif_loisir',
 'mean_classe_1',
 'mean_pointe',
 'mean_depart_we',
 'flg_track_nl_lowcost',
 'flg_track_nl',
 'days_since_last_visit',
 'tx_conversion',
 'anciennete',
 'recence_cmd',
 'AGE']

In [22]:
#continue_columns = data3.columns[1:]
#d_frame = data3
datafin2 = [data3.select(f.mean(feature)).collect()[0][0]  for feature in continuous]
datafin2

[1.515022865927351,
 1.2363232480537965,
 152.64514973822276,
 43.19983674929222,
 0.3599612090605884,
 0.20166499766517768,
 0.3525474532671343,
 0.321143304334669,
 0.13100876267904052,
 0.6506145379939793,
 16.574795970884495,
 0.34099569046665423,
 1707.0163204391963,
 28.963997587586228,
 36.77269796022761]

In [23]:
d_frame = data3.select([f.when(data3[feature].isNotNull(),data3[feature])\
           .otherwise(-1).alias(feature) for i,feature in enumerate(client_cols_to_keep)]\
          +[f.when(data3[feature].isNotNull(),data3[feature])\
            .otherwise(datafin2[i]).alias(feature) for i,feature in enumerate(continuous)])

In [24]:
d_frame.toPandas().head(5)

,ID_CLIENT,LBL_STATUT_CLT,LBL_GEO_AIR,LBL_SEG_COMPORTEMENTAL,LBL_GEO_TRAIN,LBL_GRP_SEGMENT_NL,LBL_SEGMENT_ANTICIPATION,FLG_CMD_CARTE_1225,nb_od,mean_nb_passagers,...,mean_classe_1,mean_pointe,mean_depart_we,flg_track_nl_lowcost,flg_track_nl,days_since_last_visit,tx_conversion,anciennete,recence_cmd,AGE
0,000843db32fbaecfbb047ca0bb04b1f9f4d9425a,Grand,Aéroports de Paris Orly,Chasseurs Bons Plans,Paris,Spectateur,Mixte,0,1.0,1.0,...,0.0,0.00,0.00,0.131009,0.650615,8.0,0.111111,1550.0,36.0,36.772698
1,001338752ea32d9de129c8f8bdf3e2224cf0bd71,Grand,Aéroport de Marseille Provence (MRS),Comportement Pro,Marseille,Spectateur,Anticipateur,0,1.0,1.0,...,1.0,0.00,0.00,0.131009,0.650615,3.0,0.130435,1667.0,25.0,35.000000
2,003fb9dca8de374386d0fa97b570950583111931,Moyen moins,Aéroport de Lyon - Saint Exupéry,Rythmes scolaires,Lyon,Spectateur,Peu Anticipateur,1,3.0,1.5,...,0.0,0.25,0.25,0.000000,1.000000,15.0,1.000000,395.0,15.0,25.000000
3,004efa6652e570ef68944b780738e159fbf2aeb5,Moyen moins,Aéroport de Lyon - Saint Exupéry,Sans contraintes,Lyon,Spectateur,Tres Anticipateur,0,1.0,1.0,...,0.5,0.00,0.50,0.131009,0.650615,17.0,0.125000,2188.0,20.0,31.000000
4,005dd0b718a8f4598ae3044e60f9c20222eb3a35,Petit,Aéroport de Strasbourg,Sans contraintes,Metz,Endormi,Non Anticipateur,0,1.0,1.0,...,0.0,0.50,0.50,0.131009,0.650615,15.0,0.333333,3005.0,15.0,32.000000


In [27]:
##Ou en utilisant une compréhension de dictionnaire
#my_dict = {data3.groupBy().avg("Age").take(1)[0][0] for i in continuous}
#print(my_dict)
#my_dict = {}
#for word in lst_words:
#    my_dict[cle(word)] = anagrammes(word)
    
#mean = data3.groupBy().avg("Age").take(1)[0][0]
#data3.withColumn("test", lit(mean)).show()

In [25]:
#from pyspark.sql.functions import lit

#dic={}
#for i in continuous:
#    dic[i]= data3.groupBy().avg(i).take(1)[0][0]
#print(dic)

In [26]:
def input_df(df):
    ds = df.select('ID_CLIENT',
    f.when(df.LBL_GEO_TRAIN.isin(['Toulouse', 'Lille', 'Dijon',
                                  'Lyon', 'Marseille', 'Paris',
                                  'Nice', 'Limoges','Rouen','Rennes',
                                  'Montpellier', 'Bordeaux', 'Metz',
                                  'Strasbourg']), df.LBL_GEO_TRAIN)\
               .otherwise('na').alias('geo_train'),
    f.when(df.LBL_GEO_AIR.isin(['Aéroports de Paris Orly',
                                'Aéroport de Bâle-Mulhouse / Bassel',
                                'Aéroport Lille Lesquin', 'Aéroport de Rennes',
                                'Aéroport de Nantes Atlantique',
                                'Aéroport de Marseille Provence  (MRS)', 
                                'Aéroport de Bordeaux Mérignac',
                                'Aéroports de Paris Roissy-Charles-de Gaulle', 
                                "Aéroport de Nice Côte d'Azur",
                                'Aéroport de Strasbourg',
                                'Aéroport de Lyon - Saint Exupéry', 
                                'Aéroport de Toulouse Blagnac']), df.LBL_GEO_AIR)\
               .otherwise('na').alias('geo_air'),
    f.when(df.FLG_CMD_CARTE_1225 == '1', '1')\
                   .otherwise('0').alias('cc_jeunes'),
    f.when(df.LBL_STATUT_CLT.isin(['Tres grand', 'Nouveau actif',
                                   'Moyen moins', ' Prospect', ' Petit',
                                   'Inactif', 'Tres petit',
                                   'Nouveau prospect', 'Moyen plus',
                                   'Grand']), df.LBL_STATUT_CLT)\
                   .otherwise('na').alias('segt_rfm'),
    f.when(df.LBL_SEGMENT_ANTICIPATION.isin(['Peu Anticipateur', 'Tres Anticipateur',
                                             'Anticipateur', 'Mixte', 'Non Anticipateur',
                                             'Non Defini']), df.LBL_SEGMENT_ANTICIPATION)\
                   .otherwise('na').alias('segt_anticipation'),
    f.when(df.LBL_SEG_COMPORTEMENTAL.isin(['Mono-commande',
                                           'Comportement Pro',
                                           'Exclusifs Agence', 
                                           'Anticipateurs Methodiques',
                                           'Chasseurs Bons Plans', 
                                           'Rythmes scolaires', 'Nouveaux',
                                           'Sans contraintes']),
           df.LBL_SEG_COMPORTEMENTAL).otherwise('na').alias('segt_comportemental'), 
    f.when(df.LBL_GRP_SEGMENT_NL.isin(['Endormi', 'Spectateur', 'Acteur',
                                       'Eteint', 'Non defini']),
           df.LBL_GRP_SEGMENT_NL).otherwise('na').alias('segt_nl'),
    f.when(((df.AGE > 0) & (df.AGE < 100)), df.AGE)\
                   .otherwise(-1).alias('age'),
    f.when(df.recence_cmd >= 0, df.recence_cmd)\
                   .otherwise(-1).alias('recence_cmd'),
    f.when(((df.mean_duree_voyage > 0) & (df.mean_duree_voyage < 750)),
           df.mean_duree_voyage).otherwise(-1).alias('mean_duree_voyage'),
    f.when(df.days_since_last_visit >= 0, df.days_since_last_visit)\
                   .otherwise(-1).alias('recence_visite'),
    f.when(df.mean_mt_voyage > 0, df.mean_mt_voyage)\
                   .otherwise(-1).alias('mean_mt_voyage'),
    f.when(df.anciennete >= 0, df.anciennete)\
                   .otherwise(-1).alias('anciennete'),
    f.when(df.nb_od > 0, df.nb_od)\
                   .otherwise(-1).alias('nb_od'),
    f.when(df.mean_nb_passagers > 0, df.mean_nb_passagers)\
                   .otherwise(-1).alias('mean_nb_passagers'),
    f.when(df.mean_tarif_loisir >= 0, df.mean_tarif_loisir)\
                   .otherwise(-1).alias('mean_tarif_loisir'),
    f.when(df.mean_classe_1 >= 0, df.mean_classe_1)\
                   .otherwise(-1).alias('mean_classe_1'),
    f.when(df.mean_pointe >= 0, df.mean_pointe)\
                   .otherwise(-1).alias('mean_pointe'),
    f.when(df.mean_depart_we >= 0, df.mean_depart_we)\
                   .otherwise(-1).alias('mean_depart_we'),
    f.when(df.tx_conversion >= 0, df.tx_conversion)\
                   .otherwise(-1).alias('tx_conversion'),
    f.when(df.flg_cmd_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_cmd_lowcost'),
    f.when(df.flg_track_nl_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_track_nl_lowcost'), 
    f.when(df.flg_track_nl == 1, '1')\
                   .otherwise('0').alias('flg_track_nl'))
    
    return ds
#df = input_df(df)

 ### Quelles sont les differentes valeurs de notre label : flg_cmd_lowcost ?

* Cette fonction retourne un dataframe en creant une colonne supplementaire
pour sauuv les valeurs presentes a chaq etapes ,puis les non presentes par les na

In [27]:
data_new = input_df(d_frame)
#data_new.limit(3).toPandas()
#modalite(data_new,"flg_cmd_lowcost")

AttributeError: 'DataFrame' object has no attribute 'flg_cmd_lowcost'

 #### Afficher l'effectif des différentes modalités de flg_cmd_lowcost du data_new

In [ ]:
data_new.groupby("flg_cmd_lowcost").agg(f.count("flg_cmd_lowcost").alias("Effectif")).show()

In [ ]:
#ds.filter(ds.flg_cmd_lowcost=='1').count()
#df.withColumn("adulte", f.when(df["age"] < 25, 0).when(df["age"] >= 25,1)).toPandas()
#d.withColumn("Total", f.count("Effectif")).show()

In [ ]:
#detail du dataframe datanew
dtail_data(data_new)

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression


### Features engineering et modélisation

In [ ]:

def preprocessed_df(df, label="flg_cmd_lowcostIndex"):
    max_values_to_define_str_cols = 10
    id_col = 'ID_CLIENT'
    
    dty = dict(df.dtypes)
    str_cols = [k for k, v in dty.items() if v == 'string']
    str_cols.remove(id_col)
    
    for c in str_cols:
        stringIndexer = StringIndexer(inputCol=c, outputCol=c+"Index")
        model_str = stringIndexer.fit(df)
        df = model_str.transform(df).drop(c)

    input_cols = df.columns
    input_cols.remove(id_col)
    input_cols.remove(label)
    
    assembler = VectorAssembler(inputCols=input_cols,
                            outputCol="features")
    df = assembler.transform(df)
    
    featureIndexer = VectorIndexer(inputCol="features", 
                   outputCol="indexedFeatures", 
                   maxCategories=max_values_to_define_str_cols).fit(df)
    return featureIndexer.transform(df), df


data, dff = preprocessed_df(data_new)



In [ ]:
dtail_data(data)

In [ ]:
dtail_data(dff) #test indexedFeatures: vector??

### Prelever un sample de data pour notre modelisation

In [ ]:
#Construction du modele avec 70% des donnees en apprentissage
Xtrain = data.sample(False, 0.7, 42)
Xval = data.subtract(Xtrain)

## Logistic regression

**Quelle est le label est renseigne pour la modelisation?**

In [ ]:
reg_log = LogisticRegression(labelCol="flg_cmd_lowcostIndex", 
                        featuresCol="indexedFeatures",elasticNetParam=0.5)

my_model = reg_log.fit(Xtrain)

my_predict = my_model.evaluate(Xval) #prediction

**Evaluer votre modele courbe roc, precision, rappel, etc....**

In [ ]:
my_predict.areaUnderROC # renvoie l AUC

**Predire alors les clients lowcoast sur un sample de data n'ayant pas servi à l'apprentissage**

## Random Forest

In [22]:
classifier = RandomForestClassifier(labelCol="flg_cmd_lowcostIndex", 
                                    featuresCol="indexedFeatures",
                                    maxDepth=15, numTrees=100)

model_rf = classifier.fit(data1)

evaluer les performance de notre modele

In [42]:
# une autre technique pour evaluer le modele

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

Evaluer soit meme le score en calculant le nombre de VP, FP, VN et FN
on calculera alors le score qui VP+VN/VP+VN+FP+FN
nb: la prediction est automatiquement creee dans le data set et correspond à la colonne prediction

In [56]:
calculer egalement le rappel et la precision

(1.0, 0.6846384667505566, 1.0, 0.9699609990872127)

In [17]:
from pyspark.ml.linalg import Vectors
scoreAndLabels = map(lambda x: (Vectors.dense([1.0 - x[0], x[0]]), x[1]),
                     [(0.1, 0.0), (0.1, 1.0), (0.4, 0.0), (0.6, 0.0), (0.6, 1.0), (0.6, 1.0), (0.8, 1.0)])
dataset = spark_session.createDataFrame(scoreAndLabels, ["raw", "label"])
dataset.head()

Row(raw=DenseVector([0.9, 0.1]), label=0.0)

## How to create a spark data frame ?

In [46]:
from pyspark.sql import Row
import datetime

l = [(datetime.date(2018,1,3), 'Ankit',25,'Female'),
     (datetime.date(2018,2,3), 'Jalfaizy',22,'Female'),
     (datetime.date(2018,1,5), 'saurabh',20,'Female'),
     (datetime.date(2018,1,12), 'Bala',26,'Female'),
     (datetime.date(2018,7,9), 'Jules',19,'Female') ,
     (datetime.date(2018,3,18), 'Arild',43,'Male'),
     (datetime.date(2018,1,5), 'sarah',20,'Male'),
     (datetime.date(2018,8,12), 'Boly',33,'Male'),
     (datetime.date(2018,4,6), 'Anita',35,'Male'),
     (datetime.date(2018,12,6), 'Jules',22,'Male'),
     (datetime.date(2018,7,24), 'Soul',20,'Female'),
     (datetime.date(2018,6,17), 'Gral',54,'Female'),
     (datetime.date(2018,9,7), 'Apoh',18,'Female'),
     (datetime.date(2018,10,4), 'Dony',32,'Female'),
     (datetime.date(2018,2,5), 'Tanoh',31,'Female'),
     (datetime.date(2018,11,12), 'Issouf',27,'Male'),
     (datetime.date(2018,10,3), 'Bilé',29,'Male'),
     (datetime.date(2018,5,3), 'Gagnon',20,'Male'),
     (datetime.date(2018,3,5), 'Papiss',28,'Male'),
     (datetime.date(2018,2,12), 'Kravitz',34,'Male'),
     (datetime.date(2018,5,9), 'Mouli',35,'Female'),
     (datetime.date(2018,8,3), 'Jacques',27,'Female'),
     (datetime.date(2018,12,5), 'soum',22,'Female'),
     (datetime.date(2018,4,12), 'MBra',36,'Female'),]

rdd = spark_session.sparkContext.parallelize(l)
people = rdd.map(lambda x: Row(date=x[0], name=x[1], age=int(x[2]),sexe=x[3]))
schemaPeople = spark_session.createDataFrame(people)
schemaPeople.toPandas()


,age,date,name,sexe
0,25,2018-01-03,Ankit,Female
1,22,2018-02-03,Jalfaizy,Female
2,20,2018-01-05,saurabh,Female
3,26,2018-01-12,Bala,Female
4,19,2018-07-09,Jules,Female
5,43,2018-03-18,Arild,Male
6,20,2018-01-05,sarah,Male
7,33,2018-08-12,Boly,Male
8,35,2018-04-06,Anita,Male
9,22,2018-12-06,Jules,Male


1- compter le nombre de personne totale 
2- compter le nombre de fille et de garcon
3- quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille)
4 - quelle est la date de dernière visite de chaque client par rapport à la date d'aujourd'hui (la colonne date correspond à la date de visite)
5 - quels sont les personnes qui ont une date de visite < 400 jours.

 ### 1- compter le nombre de personne totale

In [47]:
schemaPeople.count()

24

 ### 2- Compter le nombre de fille et de garcon

In [48]:
schemaPeople.groupby("sexe").agg(f.count("sexe").alias("Effectif")).show()

+------+--------+
|  sexe|Effectif|
+------+--------+
|Female|      14|
|  Male|      10|
+------+--------+



 ### 3- Quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille) ?

In [49]:
#df.groupby(["sexe"]).describe()
schemaPeople.groupby("sexe").agg(
     f.count("sexe").alias("Effectif"),
     f.mean("age").alias("Moy"),  
     f.min("age").alias("Age Min"),
     f.max("age").alias("Age Max")
).show()

+------+--------+------------------+-------+-------+
|  sexe|Effectif|               Moy|Age Min|Age Max|
+------+--------+------------------+-------+-------+
|Female|      14|27.642857142857142|     18|     54|
|  Male|      10|              29.1|     20|     43|
+------+--------+------------------+-------+-------+



In [50]:
medi = [('G',schemaPeople.filter(schemaPeople['sexe'] == 'G').approxQuantile('age',(0.5,),0.001)[0],),
('F',schemaPeople.filter(schemaPeople['sexe'] == 'F').approxQuantile('age',(0.5,),0.001)[0],)]

rdd = spark_session.sparkContext.parallelize(medi)
med = rdd.map(lambda x: Row(sexe = x[0], mediane=x[1]))
schemaMedian = spark_session.createDataFrame(med)

IndexError: list index out of range

In [ ]:
schemaMedian.join(other, on = 'sexe').show()

 ### 4 - Quelle est la durée de visite de chaque client depuis sa dernière date visite jusqu'à la date d'aujourd'hui ?

In [ ]:
dd = schemaPeople.select("*",
                         f.lit(datetime.date.today()).alias("date_max"))
dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit')).toPandas()

#dd2 = spark_session.createDataFrame(dd1)

In [ ]:
dd.groupby("name").agg(f.count("sexe").alias("Effectif")).toPandas()

In [ ]:
df = dd.select("*", (f.datediff('date_max', 'date')<400).alias('Boolean'))

df.groupby("Boolean").agg(f.count("Boolean").alias("last date ago<400")).show()

In [ ]:
dd.filter(df["sexe"].isNull()).show()

In [ ]:
dd.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in dd.columns]).show()

In [ ]:
from pyspark.sql.functions import isnan
dd.filter((dd["sexe"] == "") | dd["sexe"].isNull() | isnan(dd["sexe"])).count()

In [ ]:
dd.select([f.count(f.when(f.isnan(c), c)).alias("missing_"+ c)
                    for c in dd.columns if c not in ["date", "date_max"] ]).show()

 ### 7 -Ajout de colonne correspondant a chaque mois de chaque ind

In [ ]:
df2 = schemaPeople.select("*", f.month('date').alias('mois'))
df2.toPandas()

 ### 8- Créer une colonne adulte qui prendra la valeur 0 pour les personnes de moins de 25 ans et 1 dans le cas contraire

In [ ]:
df.withColumn("adulte", f.when(df["age"] < 25, 0).when(df["age"] >= 25,1)).toPandas()